In [41]:
import asyncio

from langchain_community.utilities.nvidia_riva import AudioStream
import pywav 
import sounddevice as sd
import numpy as np
import wave

In [14]:
audio_file = "slum.wav"
wav_file = pywav.WavRead(audio_file)
audio_data = wav_file.getdata()
sample_rate = wav_file.getsamplerate()
delay_time = 1 / 4
chunk_size = int(sample_rate * delay_time)
delay_time = 1 / 8
num_channels = wav_file.getnumofchannels()

In [44]:
def chunk_audio(audio_array, n):
    chunk_size = len(audio_array) // n
    return [audio_array[i * chunk_size : (i + 1) * chunk_size] for i in range(n)]

# Step 1: Load WAV file
with wave.open(audio_file, "rb") as wf:
    channels = wf.getnchannels()
    sample_width = wf.getsampwidth()
    framerate = wf.getframerate()
    total_frames = wf.getnframes()
    audio_bytes = wf.readframes(total_frames)

# Step 2: Convert to NumPy array
dtype_map = {1: np.int8, 2: np.int16, 4: np.int32}
dtype = dtype_map[sample_width]

audio_array = np.frombuffer(audio_bytes, dtype=dtype)
    
# audio_chunks = [
#     audio_data[0 + i : chunk_size + i] for i in range(0, len(audio_data), chunk_size)
# ]

audio_chunks = chunk_audio(audio_array, 8)

async def producer(input_stream) -> None:
    """Produces audio chunk bytes into an AudioStream as streaming audio input."""
    for chunk in audio_chunks:
        await input_stream.aput(chunk)
    input_stream.close()


async def consumer(input_stream, output_stream) -> None:
    """
    Consumes audio chunks from input stream and passes them along the chain
    constructed comprised of ASR -> text based prompt for an LLM -> TTS chunks
    with synthesized voice of LLM response put in an output stream.
    """
    while not input_stream.complete:
         async for chunk in input_stream:
       # async for chunk in chain.astream(input_stream):
            await output_stream.put(
                chunk
            )  # for production code don't forget to add a timeout


input_stream = AudioStream(maxsize=1000)
output_stream = asyncio.Queue()

# send data into the chain
producer_task = asyncio.create_task(producer(input_stream))
# get data out of the chain
consumer_task = asyncio.create_task(consumer(input_stream, output_stream))
gen_audio_last = 0
while not consumer_task.done():
    try:

        generated_audio = await asyncio.wait_for(
            output_stream.get(), timeout=2
        )  # for production code don't forget to add a timeout
            
        if generated_audio != gen_audio_last:
            gen_audio_last = generated_audio
            sd.play(generated_audio, samplerate=framerate)
            # audio_array = np.frombuffer(generated_audio, dtype=np.int16)
            # #audio_array = np.frombuffer(generated_audio, dtype=np.int16)
            # sd.play(audio_array, samplerate=sample_rate)
        #sd.play(generated_audio, samplerate=sample_rate)

    except asyncio.TimeoutError:
        continue

await producer_task
await consumer_task

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [55]:
# Load audio from file

audio_file = "slum.wav"
chunk_size = 4096 

# Step 1: Load WAV file
with wave.open(audio_file, "rb") as wf:
    channels = wf.getnchannels()
    sample_width = wf.getsampwidth()
    print(f"Sample width: {sample_width}")
    framerate = wf.getframerate()
    total_frames = wf.getnframes()
    audio_bytes = wf.readframes(total_frames)

audio_chunks = [
    audio_bytes[i : i + chunk_size]
    for i in range(0, len(audio_bytes), chunk_size)
]


with sd.OutputStream(samplerate=16000, channels=1, dtype='int16') as stream:
    for chunk in audio_chunks:
        audio_np = np.frombuffer(chunk, dtype=np.int16)  # if 16-bit mono PCM
        #sd.play(audio_np, samplerate=framerate)
        stream.write(audio_np)

            
    #sd.wait()
# raw_audio = b"".join(audio_chunks)
# audio_np = np.frombuffer(raw_audio, dtype=np.int16)
# sd.play(audio_np, samplerate=framerate)
# sd.wait()

Error: file does not start with RIFF id

In [60]:
from ttstokenizer import TTSTokenizer

tokenizer = TTSTokenizer()
print(len(tokenizer("Text to tokenize")))

14
